[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/anymap-ts/blob/main/docs/maplibre/zarr_layer.ipynb)
[![Open in Notebook.link](https://notebook.link/static/badge.svg)](https://notebook.link/github/opengeos/anymap-ts/tree/main/lab/?path=docs/maplibre/zarr_layer.ipynb)

# Zarr Layer Example

This notebook demonstrates how to visualize Zarr datasets using the @carbonplan/zarr-layer package.

The Zarr layer provides GPU-accelerated rendering with support for:
- Zarr v2 and v3 formats via zarrita.js
- EPSG:4326 and EPSG:3857 projections
- Globe and mercator projections
- Multi-dimensional data with selectors
- Dynamic updates (colormap, clim, selector)

## Basic Example: 4D Climate Pyramid

Add a Zarr layer using the CarbonPlan 4D climate pyramid dataset with temperature and precipitation by month.

In [ ]:
from anymap_ts import Map

# CarbonPlan 4D climate pyramid (v2, temp/precip by month)
ZARR_URL = (
    "https://carbonplan-maps.s3.us-west-2.amazonaws.com/v2/demo/4d/tavg-prec-month"
)

# Create a MapLibre map
m = Map(center=[-100, 40], zoom=3, style="dark-matter")

# Add the Zarr layer with precipitation data for January
# Note: This dataset uses 'x' and 'y' instead of 'lon' and 'lat'
m.add_zarr_layer(
    url=ZARR_URL,
    variable="climate",
    name="climate-prec",
    clim=(0, 300),  # Precipitation range
    colormap=[
        "#f7fbff",
        "#deebf7",
        "#c6dbef",
        "#9ecae1",
        "#6baed6",
        "#4292c6",
        "#2171b5",
        "#08519c",
        "#08306b",
    ],  # Blues
    opacity=0.8,
    selector={"band": "prec", "month": 1},  # January precipitation
    spatial_dimensions={"lat": "y", "lon": "x"},  # Custom dimension names
    zarr_version=2,
)

m

## Update Layer Properties

You can dynamically update layer properties like selector, colormap, and clim without recreating the layer.

In [ ]:
# Update to show July precipitation
m.update_zarr_layer(
    layer_id="climate-prec",
    selector={"band": "prec", "month": 7},  # July
)

In [ ]:
# Switch to temperature data with a warm colormap
m.update_zarr_layer(
    layer_id="climate-prec",
    selector={"band": "tavg", "month": 7},  # July temperature
    clim=(-20, 30),  # Temperature range in Celsius
    colormap=[
        "#313695",
        "#4575b4",
        "#74add1",
        "#abd9e9",
        "#e0f3f8",
        "#ffffbf",
        "#fee090",
        "#fdae61",
        "#f46d43",
        "#d73027",
        "#a50026",
    ],
)

## Ocean Temperature (Zarr v3)

This example uses a Zarr v3 pyramid dataset with ocean temperature data.

In [ ]:
from anymap_ts import Map

# Ocean temperature (v3 pyramid, EPSG:3857)
OCEAN_URL = "https://atlantis-vis-o.s3-ext.jc.rl.ac.uk/noc-npd-era5-demo/npd-eorca1-era5v1/gn/T1y/tos_con"

m2 = Map(center=[0, 30], zoom=2, style="dark-matter")

m2.add_zarr_layer(
    url=OCEAN_URL,
    variable="tos_con",
    name="ocean-temp",
    clim=(0, 50),
    colormap=[
        "#000004",
        "#1b0c41",
        "#4a0c6b",
        "#781c6d",
        "#a52c60",
        "#cf4446",
        "#ed6925",
        "#fb9b06",
        "#f7d13d",
        "#fcffa4",
    ],  # Inferno
    opacity=0.9,
    selector={"time": 0},
)

m2

## Burn Probability over CONUS

This example uses a high-resolution (30m) Zarr v3 dataset showing wildfire burn probability.

In [ ]:
from anymap_ts import Map

# Burn Probability over CONUS (30m resolution, Zarr v3)
BURN_URL = "https://carbonplan-share.s3.us-west-2.amazonaws.com/zarr-layer-examples/13-lvl-30m-4326-scott-BP.zarr"

m3 = Map(center=[-98, 39], zoom=4, style="dark-matter")

# Note: This dataset uses "latitude"/"longitude" dimension names (not "lat"/"lon")
# The library will read bounds from the coordinate arrays automatically
m3.add_zarr_layer(
    url=BURN_URL,
    variable="BP",
    name="burn-prob",
    clim=(0, 0.13),
    colormap=[
        "#000004",
        "#1b0c41",
        "#4a0c6b",
        "#781c6d",
        "#a52c60",
        "#cf4446",
        "#ed6925",
        "#fb9b06",
        "#f7d13d",
        "#fcffa4",
    ],  # Inferno/Fire
    opacity=0.9,
    spatial_dimensions={
        "lat": "latitude",
        "lon": "longitude",
    },  # Custom dimension names
    zarr_version=3,
)

m3

## Configuration Options

The `add_zarr_layer` method supports several configuration options:

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `url` | str | required | URL to the Zarr store |
| `variable` | str | required | Variable name in the Zarr dataset |
| `name` | str | auto | Layer identifier |
| `colormap` | List[str] | ['#000000', '#ffffff'] | Hex color strings for visualization |
| `clim` | Tuple[float, float] | (0, 100) | Color scale limits (min, max) |
| `opacity` | float | 1.0 | Layer opacity (0-1) |
| `selector` | Dict | {} | Dimension selector (e.g., {"month": 4, "band": "tavg"}) |
| `minzoom` | int | 0 | Minimum zoom level for rendering |
| `maxzoom` | int | 22 | Maximum zoom level for rendering |
| `fill_value` | float | auto | No-data value |
| `spatial_dimensions` | Dict | auto | Custom spatial dimension names (e.g., {"lat": "y", "lon": "x"}) |
| `zarr_version` | int | auto | Zarr format version (2 or 3) |
| `bounds` | List[float] | auto | Explicit bounds [west, south, east, north] for datasets without coordinate arrays |

## Sample Datasets

CarbonPlan provides several sample datasets at [zarr-layer.demo.carbonplan.org](https://zarr-layer.demo.carbonplan.org/):

| Dataset | URL | Variable | Description |
|---------|-----|----------|-------------|
| 4D Climate | carbonplan-maps.s3.../v2/demo/4d/tavg-prec-month | climate | Temp/precip by month |
| Ocean Temperature | atlantis-vis-o.s3-ext.../tos_con | tos_con | v3 pyramid, EPSG:3857 |
| Burn Probability | carbonplan-share.s3.../13-lvl-30m-4326-scott-BP.zarr | BP | 30m resolution CONUS |

In [ ]:
# Remove a zarr layer
m.remove_zarr_layer("climate-prec")